In [ ]:
import sys
from pathlib import Path

import numpy as np
from aiida import load_profile
from aiida_workgraph import WorkGraph

import calculations

In [ ]:
sys.tracebacklimit = None

In [ ]:
_ = load_profile()

# Define parameters

In [ ]:
debug = False
use_mpi = False

nn = 1  # number of nearest neighbors | don't use 0!

input_path = Path("output/BLG/AB/rectangular")

# Structure

## lengths
lx = 50  # length in x direction (Å)
ly = lx / np.sqrt(3)  # length in y direction (Å) keeping the b/a ratio
lz = 10  # length in z direction (Å)
workdir = input_path / f"len_{lx}x{int(ly)}/nn_{nn}"

## or, repetitions
nx = 1  # number of repetitions in x direction
ny = 1  # number of repetitions in y direction
nz = 1  # number of repetitions in z direction
workdir = input_path / f"rep_{nx}x{ny}/nn_{nn}"

workdir.mkdir(parents=True, exist_ok=True)

## distances
distances = [0.0, 1.425, 2.468, 2.850]

# Hamiltonian
hopping_parameters = [0.0, -2.7, 0.0, -0.27]
interlater_coupling = 0.33

# potential
alpha = [1.0, 0.5]
onsite_term = 0.0
potential_type = "null"
potential_params = {
    "amplitude": 1.0,
    # "width": 0.5,
    # "height": 0.5,
}

paths = {
    "input_path": input_path.as_posix(),
    "output_path": workdir.as_posix(),
}

band_params = {
    "high_sym_points": {
        "Γ": (0.00000, 0.00000, 0.00000),
        "P": (0.00000, 0.33333, 0.00000),
        "X": (0.00000, 0.50000, 0.00000),
        "W": (0.50000, 0.50000, 0.00000),
        "Y": (0.50000, 0.00000, 0.00000),
    },
    "k_path": "Γ P X W Y Γ W",
    "points_per_segment": 100,
    "use_sparse_solver": False,
    "sparse_solver_params": {"k": 3, "sigma": 1e-8},
    "use_mpi": use_mpi,
    "fig_filename": "bands.png",
}

In [ ]:
wg = WorkGraph("2D_potential_band_structure")
wg.add_task(
    calculations.define_structure,
    name="define_structure",
    paths=paths,
    repetitions=[nx, ny, nz],
)
wg.add_task(
    calculations.build_hamiltonian,
    name="build_hamiltonian",
    structure=wg.tasks["define_structure"].outputs["result"],
    workdir=paths["output_path"],
    nearest_neighbor=nn,
    distances=distances,
    hopping_parameters=hopping_parameters,
    interlater_coupling=interlater_coupling,
)
wg.add_task(
    calculations.apply_onsite_term,
    name="apply_onsite_term",
    H_file=wg.tasks["build_hamiltonian"].outputs["result"],
    potential_type=potential_type,
    potential_params=potential_params,
    workdir=paths["output_path"],
    onsite_term=onsite_term,
    alpha=alpha,
)
wg.add_task(
    calculations.get_band_structure,
    name="get_band_structure",
    H_file=wg.tasks["apply_onsite_term"].outputs["H_file"],
    workdir=wg.tasks["apply_onsite_term"].outputs["workdir"],
    band_params=band_params,
)
node = wg.run()